Initial read <br>
The delimiter is ';'

In [7]:
import pandas as pd

csv = pd.read_csv("dataset.csv", sep=';')

Remove all comment columns since they're not data

In [8]:
csv = csv.drop("Cancellation comments", axis=1)
csv = csv.drop("Departure delay comments", axis=1)
csv = csv.drop("Arrival delay comments", axis=1)

Remove all duplicates

In [9]:
csv = csv.drop_duplicates()

Clean Date column

In [10]:
csv["Date"] = pd.to_datetime(csv["Date"], errors="coerce")
csv.dropna(subset="Date", inplace=True)
out_of_range_dates = csv[(csv.Date < "2000-01-01") | (csv.Date > "2025-01-01")]
csv.drop(out_of_range_dates.index, inplace=True)


Clean Service column

In [11]:
csv["Service"] = csv["Service"].convert_dtypes()

Clean Departure station

In [12]:
csv = csv.dropna(subset="Departure station")
csv["Departure station"] = csv["Departure station"].convert_dtypes()

Clean Arrival station

In [13]:
csv = csv.dropna(subset="Arrival station")
csv["Arrival station"] = csv["Arrival station"].convert_dtypes()

Clean Average journey time

In [14]:
csv = csv.dropna(subset="Average journey time")
csv["Average journey time"] = csv["Average journey time"].convert_dtypes()

Clean Number of scheduled trains

In [15]:
csv = csv.dropna(subset="Number of scheduled trains")
csv["Number of scheduled trains"] = csv["Number of scheduled trains"].convert_dtypes()

Clean Number of cancelled trains

In [16]:
csv = csv.dropna(subset="Number of cancelled trains")
csv["Number of cancelled trains"] = csv["Number of cancelled trains"].convert_dtypes()

Clean Number of trains delayed at departure

In [17]:
csv = csv.dropna(subset="Number of trains delayed at departure")
csv["Number of trains delayed at departure"] = csv["Number of trains delayed at departure"].convert_dtypes()

Clean Average delay of late trains at departure

In [18]:
csv = csv.dropna(subset="Average delay of late trains at departure")
csv["Average delay of late trains at departure"] = csv["Average delay of late trains at departure"].convert_dtypes()

Clean Average delay of all trains at departure

In [19]:
csv = csv.dropna(subset="Average delay of all trains at departure")
csv["Average delay of all trains at departure"] = csv["Average delay of all trains at departure"].convert_dtypes()

Clean Number of trains delayed at arrival

In [20]:
csv = csv.dropna(subset="Number of trains delayed at arrival")
csv["Number of trains delayed at arrival"] = csv["Number of trains delayed at arrival"].convert_dtypes()

Clean Average delay of late trains at arrival

In [21]:
csv = csv.dropna(subset="Average delay of late trains at arrival")
csv["Average delay of late trains at arrival"] = csv["Average delay of late trains at arrival"].convert_dtypes()

Clean Average delay of all trains at arrival

In [22]:
csv = csv.dropna(subset="Average delay of all trains at arrival")
csv["Average delay of all trains at arrival"] = csv["Average delay of all trains at arrival"].convert_dtypes()

Clean Number of trains delayed > 15 min

In [23]:
csv = csv.dropna(subset="Number of trains delayed > 15min")
csv["Number of trains delayed > 15min"] = csv["Number of trains delayed > 15min"].convert_dtypes()

Clean Average delay of trains > 15min (if competing with flights)

In [24]:
csv = csv.dropna(subset="Average delay of trains > 15min (if competing with flights)")
csv["Average delay of trains > 15min (if competing with flights)"] = csv["Average delay of trains > 15min (if competing with flights)"].convert_dtypes()

Clean Number of trains delayed > 30min

In [25]:
csv = csv.dropna(subset="Number of trains delayed > 30min")
csv["Number of trains delayed > 30min"] = csv["Number of trains delayed > 30min"].convert_dtypes()

Clean Number of trains delayed > 60min

In [26]:
csv = csv.dropna(subset="Number of trains delayed > 60min")
csv["Number of trains delayed > 60min"] = csv["Number of trains delayed > 60min"].convert_dtypes()

Clean Pct delay due to external causes

In [27]:
csv = csv.dropna(subset="Pct delay due to external causes")
csv["Pct delay due to external causes"] = csv["Pct delay due to external causes"].convert_dtypes()

Clean Pct delay due to infrastructure

In [28]:
csv.dropna(subset="Pct delay due to infrastructure")
csv["Pct delay due to infrastructure"] = csv["Pct delay due to infrastructure"].convert_dtypes()

CleanPct delay due to traffic management

In [29]:
csv.dropna(subset="Pct delay due to traffic management")
csv["Pct delay due to traffic management"] = csv["Pct delay due to traffic management"].convert_dtypes()

Clean Pct delay due to rolling stock

In [30]:
csv.dropna(subset="Pct delay due to rolling stock")
csv["Pct delay due to rolling stock"] = csv["Pct delay due to rolling stock"].convert_dtypes()

Clean Pct delay due to station management and equipment reuse

In [31]:
csv = csv.dropna(subset="Pct delay due to station management and equipment reuse")
csv["Pct delay due to station management and equipment reuse"] = csv["Pct delay due to station management and equipment reuse"].convert_dtypes()

Clean Pct delay due to passenger handling (crowding, disabled persons, connections)

In [32]:
csv = csv.dropna(subset="Pct delay due to passenger handling (crowding, disabled persons, connections)")
csv["Pct delay due to passenger handling (crowding, disabled persons, connections)"] = csv["Pct delay due to passenger handling (crowding, disabled persons, connections)"].convert_dtypes()